In [1]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [3]:
import pulse_compiler_helper_fns

In [5]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [6]:
schedules = []
thetas = np.linspace(0, np.pi, 13)
for theta in thetas:
    for basis in ['X', 'Y', 'Z']:
        circuit = q.QuantumCircuit(1)
        circuit.rx(theta, 0)
        decomposed_circuit = circuit.decompose()
        pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
        transpiled_circuit = q.transpile(circuit, backend, optimization_level=1)
        schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
        if basis == 'Y':  # apply H Sdag
            schedule |= cmd_def.get('u1', qubits=[0], P0=-np.pi/2) << schedule.duration
            schedule |= cmd_def.get('u2', qubits=[0], P0=0, P1=np.pi) << schedule.duration
        elif basis == 'X':  # apply H
            schedule |= cmd_def.get('u2', qubits=[0], P0=0, P1=np.pi) << schedule.duration
        schedule |= measure << schedule.duration
        schedules.append(schedule)

In [8]:
len(schedules)

39

In [9]:
shots = 2000
job = q.execute(schedules, backend=backend, shots=shots)
print(job.job_id())

5dced8592a4d81001215b795


In [20]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [18]:
backend.status()

BackendStatus(backend_name='alt_almaden', backend_version='1.2.4', operational=True, pending_jobs=1, status_msg='active')

In [21]:
result = job.result()

In [23]:
#marginal_counts = q.ignis.verification.tomography.data.marginal_counts
for i in range(13):
    x_counts = marginal_counts(result.get_counts(3 * i), meas_qubits=[0])
    y_counts = marginal_counts(result.get_counts(3 * i + 1), meas_qubits=[0])
    z_counts = marginal_counts(result.get_counts(3 * i + 2), meas_qubits=[0])
    
    # bloch vector components:
    x = (x_counts.get('0', 0) / shots - 0.5) * 2
    y = (y_counts.get('0', 0) / shots - 0.5) * 2
    z = (z_counts.get('0', 0) / shots - 0.5) * 2
    print(thetas[i], x, y, z)

0.0 0.04699999999999993 0.09600000000000009 0.96
0.261799387799 0.028000000000000025 -0.132 0.905
0.523598775598 0.05499999999999994 -0.376 0.8520000000000001
0.785398163397 0.052000000000000046 -0.5349999999999999 0.72
1.0471975512 0.04200000000000004 -0.722 0.498
1.308996939 0.07600000000000007 -0.786 0.32400000000000007
1.57079632679 0.05400000000000005 -0.836 0.06499999999999995
1.83259571459 -0.0010000000000000009 -0.839 -0.16500000000000004
2.09439510239 0.03200000000000003 -0.739 -0.385
2.35619449019 0.07600000000000007 -0.599 -0.55
2.61799387799 0.04400000000000004 -0.392 -0.752
2.87979326579 0.04699999999999993 -0.17200000000000004 -0.794
3.14159265359 -0.01100000000000001 -0.028000000000000025 -0.841


In [22]:
from itertools import combinations
from functools import reduce
from re import match
import numpy as np

def marginal_counts(counts, meas_qubits=True, pad_zeros=False):
    """
    Compute marginal counts from a counts dictionary.

    Args:
        counts (dict): a counts dictionary.
        meas_qubits (True, list(int)): the qubits to NOT be marinalized over
            if this is True meas_qubits will be all measured qubits
            (default: True).
        pad_zeros (Bool): Include zero count outcomes in return dict.

    Returns:
        A counts dictionary for the specified qubits. The returned dictionary
        will have any whitespace trimmed from the input counts keys. Thus if
        meas_qubits=True the returned dictionary will have the same values as
        the input dictionary, but with whitespace trimmed from the keys.
    """

    # Extract total number of qubits from first count key
    # We trim the whitespace seperating classical registers
    # and count the number of digits
    num_qubits = len(next(iter(counts)).replace(' ', ''))

    # Check if we do not need to marginalize. In this case we just trim
    # whitespace from count keys
    if num_qubits == len(meas_qubits) or (meas_qubits is True):
        ret = {}
        for key, val in counts.items():
            key = key.replace(' ', '')
            ret[key] = val
        return ret

    # Sort the measured qubits into decending order
    # Since bitstrings have qubit-0 as least significant bit
    if meas_qubits is True:
        meas_qubits = range(num_qubits)  # All measured
    qs = sorted(meas_qubits, reverse=True)

    # Generate bitstring keys for measured qubits
    meas_keys = count_keys(len(qs))

    # Get regex match strings for suming outcomes of other qubits
    rgx = []
    for key in meas_keys:
        def helper(x, y):
            if y in qs:
                return key[qs.index(y)] + x
            return '\\d' + x
        rgx.append(reduce(helper, range(num_qubits), ''))

    # Build the return list
    meas_counts = []
    for m in rgx:
        c = 0
        for key, val in counts.items():
            if match(m, key.replace(' ', '')):
                c += val
        meas_counts.append(c)

    # Return as counts dict on measured qubits only
    if pad_zeros is True:
        return dict(zip(meas_keys, meas_counts))
    ret = {}
    for key, val in zip(meas_keys, meas_counts):
        if val != 0:
            ret[key] = val
    return ret


def count_keys(num_qubits):
    """
    Return ordered count keys.
    """
    return [bin(j)[2:].zfill(num_qubits)
            for j in range(2 ** num_qubits)]


In [25]:
backend

/Users/alijavadi/Desktop/ajavadia-terra/qiskit/providers/models/backendconfiguration.py:354: UserWarning: The time delta is now returned in units of [s] rather than [ns].
  warnings.warn("The time delta is now returned in units of [s] rather than [ns].")
/Users/alijavadi/Desktop/ajavadia-terra/qiskit/providers/models/backendconfiguration.py:348: UserWarning: The time delta is now returned in units of [s] rather than [ns].
  warnings.warn("The time delta is now returned in units of [s] rather than [ns].")


<IBMQBackend('alt_almaden') from IBMQ(hub='ibm-q-dev', group='qiskit-terra', project='pulse-testing')>